In [1]:
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import convnet_builder as bare_model_builder

PWD = '../../..'
sys.path.append(PWD)

In [2]:
PROJECT_NAME = 'convnet'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 4096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22134, 24), (17707, 24), (4427, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    # Convolutional network params
    hp_n_conv_blocks = hp.Int("n_conv_blocks", min_value=1, max_value=4, step=1)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8])
    hp_conv_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])

    # MLP at the end params
    hp_mlp_n_hidden_layers = hp.Int("n_mlp_hidden_layers", min_value=0, max_value=3, step=1, default=0)
    hp_mlp_units_mult, hp_mlp_dropout = None, None
    if hp_mlp_n_hidden_layers > 0:
        hp_mlp_units_mult = hp.Choice("mlp_units_mult", values=[1, 2, 4, 8, 16], default=4)
        hp_mlp_dropout = hp.Choice("mlp_dropout", values=[0.0, 0.2, 0.5])

    # Other params
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_conv_blocks, hp_n_conv_layers, hp_filters_mult, hp_conv_spatial_dropout,
                               hp_mlp_n_hidden_layers, hp_mlp_units_mult, hp_mlp_dropout, hp_batch_normalization,
                               hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 8)             32        
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 dense (Dense)               (None, 1)                 193       
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_conv_blocks': 2, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 64369
========== Model 1 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 6625
========== Model 2 ==========
{'n_conv_blocks': 4, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.5}
Number of parameters: 181153
========== Model 3 ==========
{'n_conv_blocks': 4, 'n_conv_layers': 

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_conv_blocks': 2, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 64369
Got score: 8.8352 (8.8459, 8.8244)
Got score: 8.2047 (8.1773, 8.2321)
Got score: 8.3512 (8.3112, 8.3911)
Got score: 8.4426 (8.4282, 8.4571)
Got score: 8.2429 (8.2810, 8.2047)


{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 6625
Got score: 8.8551 (8.8233, 8.8870)
Got score: 32.2325 (56.2433, 8.2217)
Got score: 15.2964 (8.3345, 22.2584)
Got score: 8.4500 (8.4443, 8.4557)
Got score: 8.2370 (8.2741, 8.1998)


{'n_conv_blocks': 4, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.5}
Number of parameters: 181153
Got score: 8.9597 (8.9795, 8.9398)
Got score: 8.3474 (8.3478, 8.3469)
Got score: 8.3663 (8.3727, 8.3599)
Got score: 8.5902 (8.5903, 8.5900)
Got score: 8.3479 (8.3449, 8.3509)


{'n_conv_blocks': 4, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 1, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 57345
Got score: 8.9799 (9.0222, 8.9377)
Got score: 8.3256 (8.3921, 8.2590)
Got score: 8.4711 (8.5804, 8.3617)
Got score: 8.5225 (8.5174, 8.5277)
Got score: 8.4444 (8.4491, 8.4398)


{'n_conv_blocks': 1, 'n_conv_layers': 3, 'conv_filters_mult': 1, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 0, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': False, 'mlp_units_mult': 16, 'mlp_dropout': 0.2}
Number of parameters: 721
Got score: 10.4756 (11.8743, 9.0768)
Got score: 182.5697 (8.3544, 356.7851)
Got score: 8.6650 (8.6441, 8.6858)
Got score: 9.6950 (9.9276, 9.4624)
Got score: 48.3030 (24.1034, 72.5025)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.42,0.23,64369
1,14.61,9.19,6625
2,8.52,0.24,181153
3,8.55,0.23,57345
4,51.94,67.01,721
